## Example 16-1

In [1]:
def simple_coroutine():
    print("-> coroutine started")
    x = yield
    print("-> coroutine received:", x)

In [6]:
my_coro = simple_coroutine()

In [7]:
my_coro

<generator object simple_coroutine at 0x05D56E30>

In [8]:
next(my_coro)

-> coroutine started


In [9]:
next(my_coro)

-> coroutine received: None


StopIteration: 

In [14]:
my_coro = simple_coroutine()

In [15]:
next(my_coro)

-> coroutine started


In [16]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [19]:
my_coro = simple_coroutine()

In [21]:
my_coro.send(None)

-> coroutine started


In [22]:
next(my_coro)

-> coroutine received: None


StopIteration: 

In [24]:
import inspect

In [25]:
help(inspect)

Help on module inspect:

NAME
    inspect - Get useful information from live Python objects.

DESCRIPTION
    This module encapsulates the interface provided by the internal special
    attributes (co_*, im_*, tb_*, etc.) in a friendlier fashion.
    It also provides some help for examining source code and class layout.
    
    Here are some of the useful functions provided by this module:
    
        ismodule(), isclass(), ismethod(), isfunction(), isgeneratorfunction(),
            isgenerator(), istraceback(), isframe(), iscode(), isbuiltin(),
            isroutine() - check object types
        getmembers() - get members of an object that satisfy a given condition
    
        getfile(), getsourcefile(), getsource() - find an object's source code
        getdoc(), getcomments() - get documentation on an object
        getmodule() - determine the module that an object came from
        getclasstree() - arrange classes so as to represent their hierarchy
    
        getargvalues(),

In [26]:
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

In [27]:
my_coro = simple_coroutine()

In [28]:
inspect.getgeneratorstate(my_coro)

'GEN_CREATED'

In [29]:
next(my_coro)

-> coroutine started


In [30]:
inspect.getgeneratorstate(my_coro)

'GEN_SUSPENDED'

In [31]:
dir(my_coro)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [32]:
def foo():
    for i in range(10):
        yield i

In [33]:
x = foo()

In [34]:
x

<generator object foo at 0x06040BF0>

In [36]:
next(x)

0

In [37]:
x.send(10)

1

In [38]:
next(x)

2

## Example 16-2

In [44]:
def simple_coroutine2(a):
    print("-> Started: a =", a)
    b = yield a
    print("-> Received: b=", b)
    c = yield a + b
    print("-> Received c =", c)

In [46]:
my_coro2 = simple_coroutine2(14)

In [47]:
my_coro2

<generator object simple_coroutine2 at 0x071C1FB0>

In [50]:
from inspect import getgeneratorstate

In [48]:
inspect.getgeneratorstate(my_coro2)

'GEN_CREATED'

In [49]:
next(my_coro2)

-> Started: a = 14


14

In [51]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [52]:
my_coro2.send(28)

-> Received: b= 28


42

In [53]:
my_coro2.send(99)

-> Received c = 99


StopIteration: 

In [54]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

## Example 16-5

In [1]:
from functools import wraps

In [2]:
def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    
    return primer

In [4]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [12]:
coro_avg = averager()

In [6]:
from inspect import getgeneratorstate

In [13]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [14]:
coro_avg.send(40)

40.0

In [15]:
coro_avg.send(50)

45.0

In [16]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [17]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

In [18]:
coro_avg.send(60)

StopIteration: 

## Example 16-13

In [10]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        item = yield
        if item is None:
            break
        total += item
        count += 1
        average = total/count
    
    return Result(count, average)

In [14]:
coro_avg = averager()

In [15]:
from inspect import getgeneratorstate

In [16]:
getgeneratorstate(coro_avg)

'GEN_CREATED'

In [17]:
next(coro_avg)

In [18]:
coro_avg.send(10)

In [19]:
coro_avg.send(30)

In [20]:
coro_avg.send(6.5)

In [21]:
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

## Example 16-17 using yield from

In [1]:
from collections import namedtuple

In [2]:
Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        item = yield average
        if item is None:
            break
        total += item
        count += 1
        average = total/count
    
    return Result(count, average)

In [3]:
# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()

In [8]:
# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
        
    report(results)

In [9]:
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

In [10]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

In [11]:
main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [14]:
averager()

<generator object averager at 0x06CEB0B0>

In [15]:
async def foo():
    pass

In [17]:
foo()

<coroutine object foo at 0x0672CF60>